In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os

# Change to your desired working directory
os.chdir('/content/drive/MyDrive/IISc/DL')


In [ ]:
from google.colab import userdata

# Note: `userdata.get` is a Colab API. If you're not using Colab, set the env
# vars as appropriate for your system.
os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')
os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')

In [2]:
os.environ["KERAS_BACKEND"] = "jax"  # Or "tensorflow" or "torch".
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "1.00"

In [3]:
!pip install -q -U keras-hub
!pip install -q -U keras

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 876.5/876.5 kB 18.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
keras-nlp 0.18.1 requires keras-hub==0.18.1, but you have keras-hub 0.21.1 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 21.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
keras-nlp 0.18.1 requires keras-hub==0.18.1, but you have keras-hub 0.21.1 which is incompatible.


In [4]:
import keras
import keras_hub

In [5]:
gemma_lm = keras_hub.models.Gemma3CausalLM.from_preset("gemma3_instruct_4b_text")

100%|██████████| 968/968 [00:00<00:00, 2.47MB/s]


100%|██████████| 3.23k/3.23k [00:00<00:00, 6.45MB/s]


100%|██████████| 4.47M/4.47M [00:00<00:00, 10.0MB/s]


100%|██████████| 7.23G/7.23G [02:24<00:00, 53.6MB/s]


In [ ]:
import zipfile
import json
import glob
import sys
import tqdm
import torch
import os

prompt_template = '''<start_of_turn>user
    Translate the following sentence to {language}.
    Text: {text}
    Only output the translated text.
    Do not include any additional text or explanations.<end_of_turn>
    <start_of_turn>model
    '''

# Language mapping function
def get_language_name(short_code):
    lang_map = {
        'ar': 'Arabic',
        'zh': 'Chinese (Traditional)',
        'fr': 'French',
        'de': 'German',
        'it': 'Italian',
        'ja': 'Japanese',
        'ko': 'Korean',
        'es': 'Spanish',
        'th': 'Thai',
        'tr': 'Turkish',
        'en': 'English',
        # Add more as needed
    }
    return lang_map.get(short_code, short_code)

# File processing setup
input_data_folder = "./data/references/validation/"
jsonl_files = glob.glob(f"{input_data_folder}/*.jsonl")
model_name = "gemma3_4b-text-keras"
output_prediction_dir = os.path.join("./data/predictions", model_name, "validation")
os.makedirs(output_prediction_dir, exist_ok=True)

BATCH_SIZE = 20  # Define your batch size

for file_path in jsonl_files:
    filename = os.path.basename(file_path)
    outfile_path = os.path.join(output_prediction_dir, filename)

    data = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line))

    results = []
    pbar = tqdm.tqdm(total=len(data))

    # Process data in batches
    for i in range(0, len(data), BATCH_SIZE):
        batch_data = data[i:i + BATCH_SIZE]
        batch_prompts = []
        batch_records = []

        for record in batch_data:
            source = record['source']
            target_locale = record['target_locale']
            target_language = get_language_name(target_locale)
            prompt = prompt_template.format(text=source, language=target_language)
            batch_prompts.append(prompt)
            batch_records.append(record) # Keep track of original records for result mapping

        # Generate translations for the batch
        # print(f"Processing batch starting at index {i}...")

        batch_outputs = gemma_lm.generate(batch_prompts, max_length=300)

        # Process each output in the batch
        for j, output in enumerate(batch_outputs):
            record = batch_records[j] # Get the corresponding record for this output
            id = record['id']
            source = record['source']
            source_locale = record['source_locale']
            source_language = get_language_name(source_locale)
            target_locale = record['target_locale']
            target_language = get_language_name(target_locale)

            translated = output.strip()

            start_tag = "<start_of_turn>model\n"
            end_tag = "<end_of_turn>"

            start_index = output.find(start_tag)
            end_index = output.find(end_tag, start_index + len(start_tag))

            if start_index != -1 and end_index != -1:
                extracted_text = output[start_index + len(start_tag):end_index]
            else:
                print(f"Tags not found in the {output}.")

            translated = extracted_text.strip()
            #print(translated)

            results.append({
                "id": id,
                "source_language": source_language,
                "target_language": target_language,
                "text": source,
                "prediction": translated,
        })

        pbar.update(len(batch_data))

        # Periodic writing
        with open(outfile_path, 'w', encoding='utf-8') as f:
            for res in results:
                f.write(json.dumps(res, ensure_ascii=False) + '\n')

    pbar.close()
    print(f"Translations saved to {outfile_path}")



  8%|▊         | 60/722 [00:35<06:28,  1.70it/s]

100%|██████████| 724/724 [04:23<00:00,  2.75it/s]


Translations saved to /content/drive/MyDrive/IISc/DL/data/predictions/gemma3_4b-text-keras/validation/fr_FR.jsonl


100%|██████████| 722/722 [04:05<00:00,  2.94it/s]


Translations saved to /content/drive/MyDrive/IISc/DL/data/predictions/gemma3_4b-text-keras/validation/zh_TW.jsonl


100%|██████████| 710/710 [03:53<00:00,  3.04it/s]


...

Translations saved to /content/drive/MyDrive/IISc/DL/data/predictions/gemma3_4b-text-keras/validation/ar_AE.jsonl


100%|██████████| 731/731 [04:11<00:00,  2.90it/s]


Translations saved to /content/drive/MyDrive/IISc/DL/data/predictions/gemma3_4b-text-keras/validation/de_DE.jsonl


100%|██████████| 739/739 [04:11<00:00,  2.93it/s]

Translations saved to /content/drive/MyDrive/IISc/DL/data/predictions/gemma3_4b-text-keras/validation/es_ES.jsonl
